In [1]:
#Import pythond library
import pandas as pd
import os
from datetime import date
import pymysql
import sys

In [2]:
#Set the working directory
os.chdir("C:/Users/Vrushabhkumar/Desktop/RVI_47/Update Data/AssetNotes")

In [3]:
#Setting connection with Mysql Server
conn = pymysql.connect(host = 'wayne-read-replica.cge5lukryrbb.us-west-2.rds.amazonaws.com',port = 3306, user='vrushabh', password='Entytle@2020')
cursor = conn.cursor()

In [4]:
#Enter jobId from Orders and Assets
orderJobId = input("Enter jobId for Orders : ")
assetJobId = input("Enter jobId for Assets : ")

Enter jobId for Orders : 14927
Enter jobId for Assets : 14928


In [5]:
Oquery = "Select distinct orderNumber from data_loader.Order where clientId = 47 and jobId in (" + orderJobId + ") and orderNumber is not null;"

In [6]:
Aquery = "Select distinct custom12 as orderNumber from data_loader.Asset where clientId = 47 and jobId in (" + assetJobId + ") and custom12 is not null;"

In [7]:
#Extract and store the orderNumber from Orders data
ON_1 = pd.read_sql_query(Oquery, conn)
ON_1 = list(map(str,ON_1['orderNumber']))

In [8]:
#Extract and store orderNumber from Assets data
ON_2 = pd.read_sql_query(Aquery, conn)
ON_2 = list(map(str,ON_2['orderNumber']))

In [9]:
#List of total destinct orderNumber from both Order and Asset
ON = ON_1 + ON_2

In [10]:
#list to string
ON = ','.join(ON)

In [11]:
#SQL query to fetch assets for which Notes to be created
query1 = "SELECT ma.id assetId, mac.custom12 orderNumber FROM platinum.MasterAsset ma \
         LEFT JOIN platinum.MasterAssetCustom mac ON \
         ma.id = mac.id AND ma.clientId = mac.clientId AND \
         ma.deleted = mac.deleted WHERE \
         ma.clientId = 47 and mac.custom12 in (" + ON + ");"

In [12]:
#Extract and store the data
df = pd.read_sql_query(query1, conn)

In [13]:
#save the data to csv file
df.to_csv("Assets.csv", index=False)

In [14]:
#Import file containing assetId and orderNumber for which AssetNotes to be created.
Asset = pd.read_csv("Assets.csv")

In [15]:
#order number of accessories corresponding to assets sold
orderNumber = list(set(Asset['orderNumber']))
orderNumber = list(map(str,orderNumber))
orderNumber = ",".join(orderNumber)

In [16]:
# Query to fetch accessories sold with assets
query2 = "Select orderNumber,itemDescription from platinum.MasterOrder where clientid = 47 and deleted = 0 and isCause = 0 and orderNumber in" + "(" + orderNumber + ");"

In [17]:
#Extract the data and store
results = pd.read_sql_query(query2, conn)
results.to_csv("Orders.csv", index=False)

In [18]:
#Import the Accessories file
Accessories = pd.read_csv("Orders.csv")

In [19]:
#Keep unique accessories in each order
Accessories.drop_duplicates(subset=['orderNumber', 'itemDescription'], inplace = True)

In [20]:
#Sort Accessories by orderNumber and then itemDescription
Accessories.sort_values(["orderNumber","itemDescription"], inplace = True)

In [21]:
#Change data type of itemDescription to str
Accessories['itemDescription'] = Accessories['itemDescription'].astype(str)

In [22]:
#Concat accessories by orderNumber
Notes = Accessories.groupby('orderNumber')['itemDescription'].apply(lambda x :"\n".join(x)).reset_index()

In [23]:
#Prepare Notes
Notes['itemDescription'] = "Accessories sold in the same order:\n-----------------------------------\n" + Notes['itemDescription']

In [24]:
#Rename column
Notes.rename(columns={'itemDescription':'notes'},inplace = True)

In [25]:
#Join Notes table with Assets table
AssetNotes = Asset.merge(Notes,how = 'left',left_on = "orderNumber",right_on = "orderNumber")

In [26]:
#Drop orderNumber column from AssetNotes table
AssetNotes.drop(["orderNumber"],axis = 1,inplace = True)

In [27]:
#Replace null values in notes with space
AssetNotes['notes'].fillna(" ",inplace = True)

In [28]:
#Fetching todays date
Tdate = date.today().strftime("%d%b%Y")

In [29]:
#Preparing name for file to be used to update AssetNotes
filename = "Updt_AssetNotes_"+Tdate+".csv"

In [30]:
#Export to csv
AssetNotes.to_csv(filename, index = False)
print("Your file *" + filename + "* is ready!!")

Your file *Updt_AssetNotes_08Sep2020.csv* is ready!!
